In [1]:
from middleware import signalHandler
from middleware.signalHandler import get_new_messages, send_response, convert_envelope, flatten_json
from middleware.signalHandler import update_groups

import json
import time
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

"""
This is silly, but the bot is listening to a chat called "Vulcan's Bar Mitzvah"...
"""

group_barmitzvah = os.getenv('group_barmitzvah')

In [3]:
from middleware.llmHandler import handle_model_interaction

In [4]:
## Handle group updates
updated_groups = update_groups()
if updated_groups != None:
    groupdict = updated_groups
#     del updated_groups

In [5]:
allowed_group_responses = [group_barmitzvah]

def messages_for_llm(df):
    """
    Filter messages to just the ones in groups the bot is allowed to respond to.
    TODO: Make this more modular
    """
    vulcan_references = df['message'].str.lower().str.contains('vulcan')
    permitted_responses = df['thread'].isin(allowed_group_responses)

    unique_new_messages = df[vulcan_references & permitted_responses]['message'].unique()
    
    return unique_new_messages

In [6]:
def reply_to_chats(channel):
    newmessages = get_new_messages()
    if newmessages != []:
        converted_new_messages = [convert_envelope(flatten_json(i)) for i in newmessages]

        while None in converted_new_messages:
            converted_new_messages.remove(None)
        if converted_new_messages == []:
            pass
        df = pd.DataFrame(converted_new_messages)

        unique_new_messages = messages_for_llm(df)
        if len(unique_new_messages) > 0:
            print(f"\n\n------\n{len(unique_new_messages)} new messages")
            for message in unique_new_messages:
                try:
                    model_response = handle_model_interaction(message)
                    formatted_response = f'>>"{message}":\n\n{model_response}'
                    print(formatted_response)

                    send_response(channel, formatted_response)
                except:
                    print("llm failed")
    else:
        converted_new_messages = []

In [ ]:
while True:
    try:
        reply_to_chats(group_barmitzvah)
    except:
        pass
    time.sleep(30)



------
1 new messages
>>"Vulcan, what's the deal with airline food?":

Arrrr, ye be wantin' ta know about the grub they be servin' on them flying machines, eh Meatbag? Well, let me tell ye, it be a mighty peculiar thing indeed. They calls 'em "meals," but I be thinkin' they be more like... experiments! Ye see, they be tryin' ta feed us somethin' that'll last fer days, like a space ration or somethin'. And
